In [1]:
! git clone https://github.com/ultralytics/yolov5.git


fatal: destination path 'yolov5' already exists and is not an empty directory.


In [2]:
cd yolov5


/content/yolov5


In [3]:
pip install -r requirements.txt

In [4]:
pip install telepot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for telepot: filename=telepot-12.7-py3-none-any.whl size=57940 sha256=a10e2528bd834002200ff33ddf3d6c516e5681bc299c56dcd18b441d47a8be91
  Stored in directory: /root/.cache/pip/wheels/9f/9a/92/2e34a093e40a09338847e54dc9fcaab4ce01a59ba17c784c6c
Successfully built telepot
ERROR: Operation cancelled by user


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
pip install telepot


  Using cached telepot-12.7-py3-none-any.whl


In [8]:
import time
import numpy as np
import cv2
import torch
import torch.backends.cudnn as cudnn
from models.experimental import attempt_load
from utils.general import non_max_suppression
import telepot

In [9]:

from models.experimental import attempt_load
from utils.general import non_max_suppression


In [10]:
import time
import numpy as np
import cv2
import torch
import torch.backends.cudnn as cudnn
from models.experimental import attempt_load
from utils.general import non_max_suppression
import telepot

In [34]:
token = 'your token' # telegram token
receiver_id = "Your bot receiver ID" # https://api.telegram.org/bot<TOKEN>/getUpdates #use this link and replace <TOKEN> with your token
camera = 0 # webcam
weights = '/content/drive/MyDrive/best_face.pt'
width, height = (640, 480) # quality


In [12]:
bot = telepot.Bot(token)
bot.sendMessage(receiver_id, 'Your camera is active now.') # send a activation message to telegram receiver id


{'message_id': 31,
 'from': {'id': 6006268132,
  'is_bot': True,
  'first_name': 'myTestBot',
  'username': 'mytest67941bot'},
 'chat': {'id': 5705723957, 'first_name': 'Pushki', 'type': 'private'},
 'date': 1715501389,
 'text': 'Your camera is active now.'}

In [13]:
device = torch.device('cpu')



In [14]:
model = torch.load(weights, map_location=device)['model'].float()  # Load the model directly using PyTorch's `torch.load()` function
stride = int(model.stride.max())

cudnn.benchmark = True

In [19]:
pip install torch --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 2.8 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.19.3
    Uninstalling nvidia-nccl-cu12-2.19.3:
      Successfully uninstalled nvidia-nccl-cu12-2.19.3
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2

In [35]:
cap = cv2.VideoCapture(camera)
cap.set(3, width) # width
cap.set(4, height) # height

False

In [20]:
import torch.nn as nn


In [21]:
for m in model.modules():
  if isinstance(m, nn.Upsample):
    m.recompute_scale_factor = None

In [36]:
while(cap.isOpened()):
    time.sleep(0.2) # wait for 0.2 second
    ret, frame_ = cap.read()
    frame = cv2.resize(frame_, (width, height), interpolation = cv2.INTER_AREA)

    if display:
        cv2.imshow("frame", frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    if ret ==True:
        img = torch.from_numpy(frame).float().to(device).permute(2, 0, 1)
        img /= 255.0  # 0 - 255 to 0.0 - 1.0

        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        pred = model(img, augment=False)[0]
        pred = non_max_suppression(pred, 0.39, 0.45, agnostic=True) # img, conf, iou, ...

        for det in pred:
            if len(det):
                conf_, class_ = det[0][4], int(det[0][5])

                if class_ == 0  and conf_ > 0.35 :
                    time_stamp = int(time.time())
                    fcm_photo = f'detected/{time_stamp}.png'
                    cv2.imwrite(fcm_photo, frame_) # notification photo
                    bot.sendPhoto(receiver_id, photo=open(fcm_photo, 'rb')) # send message to telegram
                    print(f'{time_stamp}.png has sent.')
                    time.sleep(1) # wait for 1 second. Only when it detects.
    else:
        break

cap.release()
cv2.destroyAllWindows()
